# MFG Class: Product Wheel Optimization

<br>

---

<br>

In this session, we're going to look at rudimentary methods of forecasting using a small dataset. There are 4 data entry points:

1. forecasted
2. ordered
3. scheduled
4. produced

**TIME LINE**

* Simple forecast w/ rolled up data
  * week 1
* More complex forecase w/ product data (+ whatever else)
  * week 2
* Product Wheel Optimization w/ the forecasted
  * weeks 3-4
  * which line to produce one, what bucket sizes
  * need transition times, product/line pairs (required and preferred)
  * production rates, deckle rates

**FOR NEXT TIME**

* Constrain by:
  * late orders
  * customer allocations
  * bucket sizes

<br>

---

<a name='top'></a>

# Contents

* 3.0 [Preparing Environment and Importing Data](#x.0)
  * 3.0.1 [Import Packages](#x.0.1)

<br>

---

<a name='x.0'></a>

## 3.0 Preparing Environment and Importing Data

[back to top](#top)

<a name='x.0.1'></a>

### 3.0.1 Import Packages

[back to top](#top)

In [51]:
!pip install ipytest

In [52]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
from ipywidgets import interact, interactive, widgets
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.metrics import r2_score
from scipy.stats import mode
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
import random
import ipytest
import datetime
ipytest.autoconfig()

<a name='x.0.2'></a>

### 3.0.2 Load Dataset - Drive

[back to top](#top)

In [53]:
# Sync your google drive folder
from google.colab import drive
drive.mount("/content/drive")

KeyboardInterrupt: ignored

In [ ]:
gsm = pd.read_excel("/content/drive/MyDrive/mfganalytic/mfg_product_wheel/data/"\
                    "Waynesboro Transitions.xlsx", 
                    sheet_name="Basis Wgt_Transition Time",
                    header=6, index_col=2)
gsm.head()

In [ ]:
gsm.iloc[:23,2:25]

In [ ]:
gsm = gsm.iloc[:23,2:25]
gsm

In [ ]:
gsm.index = gsm.index.astype(str)

In [ ]:
gsm.to_csv("/content/drive/MyDrive/mfganalytic/mfg_product_wheel/data/"\
                    "gsm_transitions.csv", index=True)

In [ ]:
gsm = pd.read_csv("/content/drive/MyDrive/mfganalytic/mfg_product_wheel/data/"\
                    "gsm_transitions.csv", index_col=0)

In [ ]:
gsm.columns.astype(float)

In [ ]:
gsm_dict = gsm.to_dict()

In [ ]:
tech = pd.read_excel("/content/drive/MyDrive/mfganalytic/mfg_product_wheel/data/"\
                    "Waynesboro Transitions.xlsx", 
                    sheet_name="Support - Line Changeovers",
                    header=0, index_col=1)
tech.head()

In [ ]:
tech

In [ ]:
to = tech['To'].values
time = tech['time'].values
frm = tech.index

You can make a representation with a dictionary

In [ ]:
dff = dict(dict())
for i, j, k in zip(frm,to,time):
  dff[i, j] = k

Or with a dataframe:

In [ ]:
dff = pd.DataFrame()
for i, j, k in zip(frm, to, time):
  dff.loc[i,j] = k

We will want to convert everything to the same units. Let's perform in minutes

In [ ]:
dff = dff*60

In [ ]:
dff

We also notice there is no data for SMS IRGATEC. Let's drop it for now, in the future we could impute the data, or fill some other way we see fit

In [ ]:
dff  = dff.drop(axis=0, index='SMS IRGATEC')

In our algorithm, we usually want to use the lowest level representation suitable for our purpose (a ditcionary in this case)

In [ ]:
tech = dff
tech_dic = tech.to_dict()

In [ ]:
tech.to_csv("/content/drive/MyDrive/mfganalytic/mfg_product_wheel/data/"\
                    "tech_transitions.csv", index=True)

<a name='x.0.3'></a>

### 3.0.3 Load Dataset - GitHub

[back to top](#top)

In [75]:
gsm = pd.read_csv("https://raw.githubusercontent.com/wesleybeckner/mfg_product_wheel/main/data/gsm_transitions.csv", index_col=0)
gsm.columns = gsm.columns.astype(float)
gsm

,8.0,10.0,12.0,12.5,13.0,13.5,15.0,16.0,17.0,22.0,25.0,28.0,30.0,32.0,34.0,40.0,45.0,47.5,51.0,59.0,64.4,68.0,85.0
0,,,,,,,,,,,,,,,,,,,,,,,
8.0,0.0,15.0,17.0,18.0,20.0,20.0,21.0,21.0,21.0,21.0,22.0,22.0,23.0,23.0,24.0,25.0,25.0,26.0,26.0,27.0,27.0,27.0,30.0
10.0,30.0,0.0,2.0,3.0,5.0,5.0,6.0,6.0,6.0,6.0,7.0,7.0,8.0,8.0,9.0,10.0,10.0,11.0,11.0,12.0,12.0,12.0,15.0
12.0,34.0,4.0,0.0,1.0,3.0,3.0,4.0,4.0,4.0,4.0,5.0,5.0,6.0,6.0,7.0,8.0,8.0,9.0,9.0,10.0,10.0,10.0,13.0
12.5,36.0,6.0,2.0,0.0,2.0,2.0,3.0,3.0,3.0,3.0,4.0,4.0,5.0,5.0,6.0,7.0,7.0,8.0,8.0,9.0,9.0,9.0,12.0
13.0,40.0,10.0,6.0,4.0,0.0,0.0,1.0,1.0,1.0,1.0,2.0,2.0,3.0,3.0,4.0,5.0,5.0,6.0,6.0,7.0,7.0,7.0,10.0
13.5,40.0,10.0,6.0,4.0,0.0,0.0,1.0,1.0,1.0,1.0,2.0,2.0,3.0,3.0,4.0,5.0,5.0,6.0,6.0,7.0,7.0,7.0,10.0
15.0,42.0,12.0,8.0,6.0,2.0,2.0,0.0,0.0,0.0,0.0,1.0,1.0,2.0,2.0,3.0,4.0,4.0,5.0,5.0,6.0,6.0,6.0,9.0
16.0,42.0,12.0,8.0,6.0,2.0,2.0,0.0,0.0,0.0,0.0,1.0,1.0,2.0,2.0,3.0,4.0,4.0,5.0,5.0,6.0,6.0,6.0,9.0
17.0,42.0,12.0,8.0,6.0,2.0,2.0,0.0,0.0,0.0,0.0,1.0,1.0,2.0,2.0,3.0,4.0,4.0,5.0,5.0,6.0,6.0,6.0,9.0


In [76]:
tech = pd.read_csv("https://raw.githubusercontent.com/wesleybeckner/mfg_product_wheel/main/data/tech_transitions.csv", index_col=0)
tech

,SMS,SSS,SMS Soft,SSS Soft,BICO,SAS
SMS,0.0,90.0,30.0,120.0,150.0,120.0
SSS,90.0,0.0,120.0,30.0,60.0,30.0
SMS Soft,30.0,120.0,0.0,90.0,150.0,120.0
SSS Soft,120.0,30.0,90.0,0.0,60.0,60.0
BICO,150.0,60.0,180.0,60.0,0.0,90.0
SAS,120.0,30.0,120.0,60.0,90.0,0.0


In [ ]:
gsm_dict = gsm.to_dict()
tech_dict = tech.to_dict()

### 3.0.4 Symbolic AI Solution

In [273]:
tech_seq = ['SMS', 'SMS Soft', 'SSS Soft', 'SAS', 'SSS', 'BICO']

In [274]:
orders = []
for i in tech.index:
  for j in gsm.index:
    orders.append([j,i,random.randint(1000,2000)])
odf = pd.DataFrame(orders, columns=['gsm', 'tech', 'kg'])
orders = odf.to_dict(orient='index')
orders[0]

{'gsm': 8.0, 'kg': 1044, 'tech': 'SMS'}

In [275]:
px.scatter(odf, x=odf.index, y='gsm', color='tech')

In [276]:
start = list(orders.keys())
random.shuffle(start)

In [277]:
px.scatter(odf, x=start, y='gsm', color='tech')

In [278]:
finish = []
for idx in start:
  finish.append(orders[idx])
finish = pd.DataFrame(finish)

In [279]:
finish.head()

,gsm,tech,kg
0,85.0,SMS Soft,1207
1,51.0,SAS,1471
2,47.5,SAS,1091
3,12.5,BICO,1422
4,12.0,SMS,1952


In [284]:
df_agg = finish.groupby('tech').apply(lambda x: x.sort_values('gsm')).loc[tech_seq]
finish = df_agg.reset_index(drop=True)

In [286]:
finish.head()

,gsm,tech,kg
0,8.0,SMS,1044
1,10.0,SMS,1027
2,12.0,SMS,1952
3,12.5,SMS,1263
4,13.0,SMS,1835


In [287]:
px.scatter(finish, x=finish.index, y='gsm', color='tech')

<a name='x.1'></a>

## 3.1 Product Wheel Optimization Using Genetic Algorithm, Simple Constraints, and Test Data

[back to top](#top)

In this iteration:

* Only consider transition time
  * ignore: kg lost (gsm/tech dependent)
* Orders are absolute
  * ignore: order probabilities
* Buckets have maximum size but no minimum size

<a name='x.2'></a>

## 3.2 Test and Genes

[back to top](#top)

In [130]:
# to start we need to create some test data (forecasted orders)
random.seed(42)
def my_little_orders_generator(order_count, 
                               gsms,
                               techs,
                               kgs):
  orders = []
  for i in range(order_count):
    orders.append([random.choice(gsms),
                  random.choice(techs),
                  random.randint(kgs[0], kgs[1])])
  return orders

In [131]:
orders  = my_little_orders_generator(100,
                           gsm.index,
                           tech.index,
                           [1000,2000])

In [132]:
orders[:2]

[[64.4, 'SMS', 1025], [17.0, 'SSS', 1228]]

In [133]:
orders = []
for i in tech.index:
  for j in gsm.index:
    orders.append([j,i,random.randint(1000,2000)])
print(len(orders))

In [134]:
odf = pd.DataFrame(orders, columns=['gsm', 'tech', 'kg'])
orders = odf.to_dict(orient='index')
orders[0]

{'gsm': 8.0, 'kg': 1689, 'tech': 'SMS'}

In [135]:
px.scatter(odf, x=odf.index, y='gsm', color='tech')

<a name='x.3'></a>

## 3.3 Calculating Distance

[back to top](#top)

In [85]:
# calculate cost (transition time) of genes
def get_time(productA, productB):
  tech_cost = tech_dict[productA['tech']][productB['tech']]
  gsm_cost = gsm_dict[productA['gsm']][productB['gsm']]
  tot_time = tech_cost + gsm_cost
  return tot_time

for more details on pytest parameterization explore [here](https://pytest.org/en/stable/parametrize.html)

In [ ]:
import pytest

In [ ]:
%%run_pytest --showlocals
@pytest.mark.parametrize('execution_number', range(100))
def test_time(execution_number):
  a = random.choice(orders)
  b = random.choice(orders)
  time = get_time(a,b)
  assert time >= 0

<a name='x.2'></a>

## 3.4 Fitness

[back to top](#top)

It is important to note here as we proceed that `genes` is different from `orders`. `orders` will contain the metadata of the `genes`. In a search algorithm like this, we want to pass around the smallest possible object to represent our final answer, in this case, that is simply the order numbers. We reserve the entire dictionary with the metadata as a reference object.

In [ ]:
# define the fitness of the genes
class Fitness:
  def __init__(self, totalTime):
    self.TotalTime = totalTime

  def __gt__(self, other):
    return self.TotalTime < other.TotalTime

  def __str__(self):
    return "{:0.2f}".format(self.TotalTime)

def get_fitness(genes, orders):
  fitness = 0
  for i in range(len(genes)-1):
    start = orders[genes[i]]
    end = orders[genes[i+1]]
    fitness += get_time(start, end)
  return Fitness(round(fitness, 2))


In [89]:
optimalFitness = get_fitness([i for i in orders.keys()], orders)

In [90]:
optimalFitness.TotalTime

942.0

In [ ]:
%%run_pytest --showlocals
def test_fitness():
  get_fitness(list(orders.keys()), orders)

<a name='x.5'></a>

## 3.5 Display

[back to top](#top)

for `display` we are going to output the order sequence, the total time, and the compute time. Here, `candidate` is the best viable answer that we are passing around in the engine, it is an instance of the parameter, `genes`.

In [ ]:
# define the display function
def display(candidate, startTime):
  timeDiff = datetime.datetime.now() - startTime
  print("{}\t{}\t{}".format(
      ' '.join(map(str, candidate.Genes)),
      candidate.Fitness,
      timeDiff)
  )

In [ ]:
%%run_pytest
def test_display():
  display(list(orders.keys()), datetime.datetime.now())

<a name='x.6'></a>

## 3.6 Mutate

[back to top](#top)

In [ ]:
# define how we mutate our genes
def mutate(genes, fnGetFitness):
  count = random.randint(2, len(genes))
  initialFitness = fnGetFitness(genes)
  while count > 0:
    count -= 1
    indexA, indexB = random.sample(range(len(genes)), 2)
    genes[indexA], genes[indexB] = genes[indexB], genes[indexA]
    fitness = fnGetFitness(genes)
    if fitness > initialFitness:
      return

<a name='x.7'></a>

## 3.7 Test

[back to top](#top)

In [ ]:
# define the rest of the test harness
def solve(self, orders, optimalSequence):
  geneset = [i for i in orders.keys()]

  def fnCreate():
    return random.sample(geneset, len(geneset))

  def fnDisplay(candidate):
    display(candidate, startTime)

  def fnGetFitness(genes):
    return get_fitness(genes, orders)

  def fnMutate(genes):
    mutate(genes, fnGetFitness)

  optimalFitness = fnGetFitness(optimalSequence)
  startTime = datetime.datetime.now()
  best = get_best(fnGetFitness, None, optimalFitness, None,
                  fnDisplay, fnMutate, fnCreate)
  self.assertTrue(not optimalFitness > best.Fitness)

<a name='x.8'></a>

## 3.8 Run

[back to top](#top)

In [142]:
geneset = [i for i in orders.keys()]

def fnCreate():
  return random.sample(geneset, len(geneset))

def fnDisplay(candidate):
  display(candidate, startTime)

def fnGetFitness(genes):
  return get_fitness(genes, orders)

def fnMutate(genes):
  mutate(genes, fnGetFitness)

optimalFitness = fnGetFitness(geneset)
print(optimalFitness)
startTime = datetime.datetime.now()
best = get_best(fnGetFitness, 
                None, 
                optimalFitness, 
                None,
                fnDisplay, 
                fnMutate, 
                fnCreate)

942.00
40 46 53 55 111 132 58 18 20 94 124 21 36 50 22 113 107 38 130 35 1 127 62 68 25 44 67 32 60 11 133 129 19 90 115 23 48 28 8 33 9 104 110 39 75 31 114 93 109 41 78 57 95 37 134 69 119 89 102 117 13 81 136 120 16 12 83 105 125 118 121 15 79 108 59 100 63 65 17 70 80 49 30 82 61 131 56 74 76 0 6 122 29 64 99 14 54 112 66 85 87 5 26 43 92 106 97 34 96 84 123 73 24 51 42 137 86 7 135 52 10 2 116 77 103 126 4 47 45 3 27 91 98 88 101 128 72 71	11310.00	0:00:00.000571
40 31 44 55 32 132 52 88 30 94 124 126 36 50 22 6 129 38 137 35 1 71 9 8 78 66 67 93 60 11 133 81 63 90 43 23 48 28 85 104 17 136 110 33 75 46 57 80 109 41 7 114 95 37 134 69 119 89 102 117 13 18 112 120 16 12 83 105 2 118 121 3 27 91 59 100 19 65 20 70 111 49 79 82 61 29 56 42 76 0 113 122 131 64 99 14 54 39 53 68 87 5 26 115 92 106 123 34 96 84 97 73 24 51 74 130 86 25 135 58 10 125 116 77 101 21 4 47 45 15 62 108 98 107 103 128 72 127	11223.00	0:00:00.006711
40 31 44 55 32 132 52 88 30 94 124 126 36 50 22 6 49 38 137 3

KeyboardInterrupt: ignored

<a name='x.8'></a>

## 3.8 Interpretation

[back to top](#top)

In [71]:
wheel = []
for idx in best.Genes:
  wheel.append(orders[idx])
wheel = pd.DataFrame(wheel)
wheel.to_csv('results.csv',index=True)

In [183]:
wheel = pd.read_csv('results.csv')

In [184]:
px.scatter(wheel, x=wheel.index, y='gsm', color='tech')

## 3.9 Crossover

[back to top](#top)

We're going to first define a handler for the engine (selects the two parents to perform crossover)

In [93]:
def _crossover(parentGenes, index, parents, get_fitness, crossover, mutate,
               generate_parent):
  donorIndex = random.randrange(0, len(parents))
  if donorIndex == index:
    donorIndex = (donorIndex + 1) % len(parents)
  childGenes = crossover(parentGenes, parents[donorIndex].Genes)
  if childGenes is None:
    # parent and donor are indistinguishable
    parents[donorIndex] = generate_parent()
    return mutate(parents[index])
  fitness = get_fitness(childGenes)
  return Chromosome(childGenes, fitness, Strategies.Crossover)

a handler for our call to get_best

In [ ]:
def fnCrossover(parent, donor):
  return crossover(parent, donor, fnGetFitness)

optimalFitness = Fitness(round(1851, 2))
startTime = datetime.datetime.now()
best = get_best(fnGetFitness, 
                None, 
                optimalFitness, 
                None,
                fnDisplay, 
                fnMutate, 
                fnCreate, 
                maxAge=500, 
                poolSize=25,
                crossover=fnCrossOver)

And now the actual function. 

We're going to start by identifying pairs that are the same in both sequences

In [129]:
from itertools import chain

class Pair:
  def __init__(self, node, adjacent):
    if node < adjacent:
      node, adjacent = adjacent, node
    self.Node = node
    self.Adjacent = adjacent

  def __eq__(self, other):
    return self.Node == other.Node and self.Adjacent == other.Adjacent

  def __hash__(self):
    return hash(self.Node) * 397 ^ hash(self.Adjacent)

def crossover(parentGenes, donorGenes, fnGetFitness):
  # pairs in donor including last-first
  pairs = {Pair(donorGenes[0], donorGenes[-1]): 0}
  for i in range(len(donorGenes) - 1):
    pairs[Pair(donorGenes[i], donorGenes[i+1])] = 0

  # check if there is a place to swap with parent
  tempGenes = parentGenes[:]
  if Pair(parentGenes[0], parentGenes[-1]) in pairs:
    # find a discontinuity
    found = False
    for i in range(len(parentGenes) -1):
      if Pair(parentGenes[i], parentGenes[i+1]) in pairs:
        continue
      tempGenes = parentGenes[i + 1:] + parentGenes[:i + 1]
      found = True
      break
    if not found:
      return None

  # find all the similar runs in both
  runs = [[tempGenes[0]]]
  for i in range(len(tempGenes) - 1):
    if Pair(tempGenes[i], tempGenes[i + 1]) in pairs:
      runs[-1].append(tempGenes[i + 1])
      continue
    runs.append([tempGenes[i + 1]])

  # find a reordering of runs that is better than the parent
  initialFitness = fnGetFitness(parentGenes)
  count = random.randint(2, 20)
  runIndexes = range(len(runs))
  while count > 0:
    count -= 1
    for i in runIndexes:
      if len(runs[i]) == 1:
        continue
      if random.randint(0, len(runs)) == 0:
        runs[i] = [n for n in reversed(runs[i])]

    indexA, indexB = random.sample(runIndexes, 2)
    runs[indexA], runs[indexB] = runs[indexB], runs[indexA]
    childGenes = list(chain.from_iterable(runs))
    if fnGetFitness(childGenes) > initialFitness:
      return childGenes
  return childGenes

In [127]:
donorGenes = [0,1,2,9,3,6,0]
parentGenes = [1,2,9,4,5,6,0]
tempGenes = parentGenes[:]
pairs = {Pair(donorGenes[0], donorGenes[-1]): 0}
for i in range(len(donorGenes) - 1):
  pairs[Pair(donorGenes[i], donorGenes[i+1])] = 0

In [120]:
pairs

{<__main__.Pair at 0x7f9d4a29b090>: 0,
 <__main__.Pair at 0x7f9d4a29b110>: 0,
 <__main__.Pair at 0x7f9d4a29b210>: 0,
 <__main__.Pair at 0x7f9d4a29ba90>: 0,
 <__main__.Pair at 0x7f9d4a29bd90>: 0,
 <__main__.Pair at 0x7f9d4a29be50>: 0,
 <__main__.Pair at 0x7f9d4a29bf50>: 0}

In [121]:
runs = [[tempGenes[0]]]
for i in range(len(tempGenes) - 1):
  if Pair(tempGenes[i], tempGenes[i + 1]) in pairs:
    runs[-1].append(tempGenes[i + 1])
    continue
  runs.append([tempGenes[i + 1]])

In [122]:
runs

[[1, 2, 9], [4], [5], [6, 0]]

In [128]:
crossover([0,1,2,3], [1,2,4,5], fnGetFitness)

[3, 1, 2, 0]

## 3.10 Run with Crossover

[back to top](#top)

We'll also change display to show if we used crossover or not

In [ ]:
def display(candidate, startTime):
    timeDiff = datetime.datetime.now() - startTime
    print("{}\t{}\t{}\t{}".format(
        ' '.join(map(str, candidate.Genes)),
        candidate.Fitness,
        candidate.Strategy.name,
        timeDiff))

In [143]:
geneset = [i for i in orders.keys()]

def fnCrossover(parent, donor):
  return crossover(parent, donor, fnGetFitness)

def fnCreate():
  return random.sample(geneset, len(geneset))

def fnDisplay(candidate):
  display(candidate, startTime)

def fnGetFitness(genes):
  return get_fitness(genes, orders)

def fnMutate(genes):
  mutate(genes, fnGetFitness)

optimalFitness = fnGetFitness(geneset)
print(optimalFitness)
startTime = datetime.datetime.now()
best = get_best(fnGetFitness, 
                None, 
                optimalFitness, 
                None,
                fnDisplay, 
                fnMutate, 
                fnCreate,
                maxAge=500,
                poolSize=25,
                crossover=fnCrossover)

942.00
45 20 76 83 33 80 0 84 126 12 87 135 23 118 111 136 112 46 60 36 123 122 13 16 68 75 97 124 35 41 56 21 8 82 125 128 39 28 2 78 40 32 7 18 88 121 115 15 4 129 107 48 114 17 59 9 62 102 91 22 73 67 74 90 31 3 26 94 53 96 54 38 25 66 117 19 85 92 61 103 10 43 70 58 64 119 57 79 52 14 106 108 100 101 77 69 105 132 1 71 120 50 11 130 95 127 47 55 116 110 42 29 133 6 72 5 30 104 137 89 134 131 113 63 86 34 24 44 109 81 98 49 65 93 99 37 51 27	11461.00	0:00:00.000394
57 29 40 20 65 56 103 94 69 60 35 80 12 21 67 83 16 59 101 43 87 111 76 47 115 121 32 124 49 44 0 70 79 88 99 23 53 46 6 50 134 19 22 81 24 114 11 3 104 108 30 38 15 98 106 36 58 130 31 126 42 137 54 2 91 7 52 78 89 129 39 1 55 82 112 116 4 133 107 131 125 84 75 74 113 110 62 28 71 77 73 100 9 85 48 10 123 26 127 61 95 92 90 17 18 51 72 34 68 120 118 93 33 122 86 13 14 117 8 63 128 136 109 66 5 37 119 97 45 96 132 135 41 105 102 64 27 25	10166.00	0:00:00.000933
57 45 92 77 112 11 15 35 65 52 9 62 22 117 93 5 59 40 31 48 1

KeyboardInterrupt: ignored

## 3.12 Smaller Problem

[back to top](#top)

In [247]:
orders  = my_little_orders_generator(20,
                           gsm.index,
                           tech.index,
                           [1000,2000])

In [248]:
odf = pd.DataFrame(orders, columns=['gsm', 'tech', 'kg'])
orders = odf.to_dict(orient='index')
orders[0]

{'gsm': 40.0, 'kg': 1623, 'tech': 'SAS'}

In [249]:
px.scatter(odf, x=odf.index, y='gsm', color='tech',
           title="Total Transition Time: {}".
           format(fnGetFitness([i for i in odf.index]).TotalTime))

In [250]:
odf = odf.reset_index()
df_agg = odf.groupby('tech').apply(lambda x: x.sort_values('gsm'))
finish = df_agg.reset_index(drop=True)

In [252]:
px.scatter(finish, x=finish.index, y='gsm', color='tech',
           title="Total Transition Time: {}".
           format(fnGetFitness([i for i in finish['index'].values]).TotalTime))

In [253]:
geneset = [i for i in orders.keys()]
optimalFitness = fnGetFitness([i for i in finish['index'].values])
print(optimalFitness)
startTime = datetime.datetime.now()
best = get_best(fnGetFitness, 
                None, 
                optimalFitness, 
                None,
                fnDisplay, 
                fnMutate, 
                fnCreate,
                maxAge=500,
                poolSize=25,
                crossover=fnCrossover)

591.00
3 16 2 19 15 14 8 11 12 18 13 0 10 9 4 1 6 5 17 7	1651.00	0:00:00.000289
3 8 13 0 14 1 9 12 2 10 15 4 17 5 16 19 7 11 6 18	1515.00	0:00:00.000471
6 1 7 12 16 5 14 0 11 19 10 8 15 17 13 9 4 3 18 2	1473.00	0:00:00.000613
6 12 2 15 8 4 19 9 14 17 5 1 10 13 16 11 7 3 0 18	1340.00	0:00:00.000879
13 9 6 19 14 8 2 17 0 4 1 3 7 16 10 18 15 5 12 11	1287.00	0:00:00.001015
12 15 8 2 10 17 16 5 11 13 1 0 3 7 9 18 14 4 19 6	1279.00	0:00:00.001749
7 8 2 17 13 11 18 0 14 4 5 16 10 12 6 3 15 19 1 9	1178.00	0:00:00.003771
12 3 8 2 13 17 16 5 11 18 14 4 0 15 7 9 10 1 19 6	1165.00	0:00:00.004285
9 3 8 2 13 5 16 17 11 18 14 4 0 15 7 12 10 1 19 6	1078.00	0:00:00.007081
4 13 11 17 0 1 18 16 5 12 14 6 19 9 15 8 2 7 10 3	1065.00	0:00:00.016071
9 17 8 2 13 5 16 0 11 18 14 4 3 15 7 12 10 1 19 6	1033.00	0:00:00.016967
3 9 1 19 14 6 11 0 4 16 7 2 15 12 5 17 18 8 13 10	1023.00	0:00:00.071802
9 18 4 3 13 17 0 10 11 14 8 2 15 7 12 5 16 1 19 6	1021.00	0:00:00.075668
19 2 7 11 18 14 6 1 5 4 17 9 0 13 3 8 10 12 

In [254]:
gnx = []
for gene in best.Genes:
  gnx.append(orders[gene])
gnx = pd.DataFrame(gnx)

In [256]:
px.scatter(gnx, x=gnx.index, y='gsm', color='tech',
           title="Total Transition Time: {}".
           format(fnGetFitness([i for i in best.Genes]).TotalTime))

## 3.12 Other Sub Problems

[back to top](#top)

what is the optimal Tech sequence?

In [257]:
# calculate cost (transition time) of genes
def get_time(productA, productB):
  tech_cost = tech_dict[productA['tech']][productB['tech']]
  gsm_cost = 0#gsm_dict[productA['gsm']][productB['gsm']]
  tot_time = tech_cost + gsm_cost
  return tot_time

In [258]:
orders = []
for i in tech.index:
  orders.append([0,i,random.randint(1000,2000)])

In [260]:
odf = pd.DataFrame(orders, columns=['gsm', 'tech', 'kg'])
orders = odf.to_dict(orient='index')
orders[0]

{'gsm': 0, 'kg': 1395, 'tech': 'SMS'}

In [262]:
geneset = [i for i in orders.keys()]
optimalFitness = Fitness(round(0, 2))
print(optimalFitness)
startTime = datetime.datetime.now()
best = get_best(fnGetFitness, 
                None, 
                optimalFitness, 
                None,
                fnDisplay, 
                fnMutate, 
                fnCreate,
                maxAge=500,
                poolSize=25,
                crossover=fnCrossover)

0.00
5 3 1 0 2 4	390.00	0:00:00.000223
5 4 1 3 0 2	330.00	0:00:00.000495
3 4 1 5 2 0	300.00	0:00:00.000946
0 2 3 5 1 4	270.00	0:00:00.006851


KeyboardInterrupt: ignored

In [271]:
tech_seq = odf.iloc[[0, 2, 3, 5, 1, 4]]['tech'].values
tech_seq = ['SMS', 'SMS Soft', 'SSS Soft', 'SAS', 'SSS', 'BICO']

array(['SMS', 'SMS Soft', 'SSS Soft', 'SAS', 'SSS', 'BICO'], dtype=object)

## 3.13 Genetic

[back to top](#top)

In [ ]:
import random
import statistics
import sys
import time
from bisect import bisect_left
from enum import Enum
from math import exp


def _generate_parent(length, geneSet, get_fitness):
    genes = []
    while len(genes) < length:
        sampleSize = min(length - len(genes), len(geneSet))
        genes.extend(random.sample(geneSet, sampleSize))
    fitness = get_fitness(genes)
    return Chromosome(genes, fitness, Strategies.Create)


def _mutate(parent, geneSet, get_fitness):
    childGenes = parent.Genes[:]
    index = random.randrange(0, len(parent.Genes))
    newGene, alternate = random.sample(geneSet, 2)
    childGenes[index] = alternate if newGene == childGenes[index] else newGene
    fitness = get_fitness(childGenes)
    return Chromosome(childGenes, fitness, Strategies.Mutate)


def _mutate_custom(parent, custom_mutate, get_fitness):
    childGenes = parent.Genes[:]
    custom_mutate(childGenes)
    fitness = get_fitness(childGenes)
    return Chromosome(childGenes, fitness, Strategies.Mutate)


def _crossover(parentGenes, index, parents, get_fitness, crossover, mutate,
               generate_parent):
    donorIndex = random.randrange(0, len(parents))
    if donorIndex == index:
        donorIndex = (donorIndex + 1) % len(parents)
    childGenes = crossover(parentGenes, parents[donorIndex].Genes)
    if childGenes is None:
        # parent and donor are indistinguishable
        parents[donorIndex] = generate_parent()
        return mutate(parents[index])
    fitness = get_fitness(childGenes)
    return Chromosome(childGenes, fitness, Strategies.Crossover)


def get_best(get_fitness, targetLen, optimalFitness, geneSet, display,
             custom_mutate=None, custom_create=None, maxAge=None,
             poolSize=1, crossover=None):
    if custom_mutate is None:
        def fnMutate(parent):
            return _mutate(parent, geneSet, get_fitness)
    else:
        def fnMutate(parent):
            return _mutate_custom(parent, custom_mutate, get_fitness)

    if custom_create is None:
        def fnGenerateParent():
            return _generate_parent(targetLen, geneSet, get_fitness)
    else:
        def fnGenerateParent():
            genes = custom_create()
            return Chromosome(genes, get_fitness(genes), Strategies.Create)

    strategyLookup = {
        Strategies.Create: lambda p, i, o: fnGenerateParent(),
        Strategies.Mutate: lambda p, i, o: fnMutate(p),
        Strategies.Crossover: lambda p, i, o:
        _crossover(p.Genes, i, o, get_fitness, crossover, fnMutate,
                   fnGenerateParent)
    }

    usedStrategies = [strategyLookup[Strategies.Mutate]]
    if crossover is not None:
        usedStrategies.append(strategyLookup[Strategies.Crossover])

        def fnNewChild(parent, index, parents):
            return random.choice(usedStrategies)(parent, index, parents)
    else:
        def fnNewChild(parent, index, parents):
            return fnMutate(parent)

    for improvement in _get_improvement(fnNewChild, fnGenerateParent,
                                        maxAge, poolSize):
        display(improvement)
        f = strategyLookup[improvement.Strategy]
        usedStrategies.append(f)
        if not optimalFitness > improvement.Fitness:
            return improvement


def _get_improvement(new_child, generate_parent, maxAge, poolSize):
    bestParent = generate_parent()
    yield bestParent
    parents = [bestParent]
    historicalFitnesses = [bestParent.Fitness]
    for _ in range(poolSize - 1):
        parent = generate_parent()
        if parent.Fitness > bestParent.Fitness:
            yield parent
            bestParent = parent
            historicalFitnesses.append(parent.Fitness)
        parents.append(parent)
    lastParentIndex = poolSize - 1
    pindex = 1
    while True:
        pindex = pindex - 1 if pindex > 0 else lastParentIndex
        parent = parents[pindex]
        child = new_child(parent, pindex, parents)
        if parent.Fitness > child.Fitness:
            if maxAge is None:
                continue
            parent.Age += 1
            if maxAge > parent.Age:
                continue
            index = bisect_left(historicalFitnesses, child.Fitness, 0,
                                len(historicalFitnesses))
            proportionSimilar = index / len(historicalFitnesses)
            if random.random() < exp(-proportionSimilar):
                parents[pindex] = child
                continue
            bestParent.Age = 0
            parents[pindex] = bestParent
            continue
        if not child.Fitness > parent.Fitness:
            # same fitness
            child.Age = parent.Age + 1
            parents[pindex] = child
            continue
        child.Age = 0
        parents[pindex] = child
        if child.Fitness > bestParent.Fitness:
            bestParent = child
            yield bestParent
            historicalFitnesses.append(bestParent.Fitness)


class Chromosome:
    def __init__(self, genes, fitness, strategy):
        self.Genes = genes
        self.Fitness = fitness
        self.Strategy = strategy
        self.Age = 0


class Strategies(Enum):
    Create = 0,
    Mutate = 1,
    Crossover = 2